# Task 1: Third-order letter approximation model



#### 1.1 Converting texts to appropriate format (Only including ASCII letters, full stops and spaces)

In [ ]:
import os
from pathlib import Path
import re

def process_text(filename):
    input_dir = Path("unprocessedTexts")
    output_dir = Path("processedTexts")
    input_dir.mkdir(exist_ok=True)
    output_dir.mkdir(exist_ok=True)
    
    try:
        # Read input file
        input_path = input_dir / filename
        with open(input_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        # Keep only letters, spaces and full stops
        processed = ''
        for char in text:
            if char.isalpha() or char == '.' or char == ' ':
                processed += char
        
        # Remove multiple spaces
        processed = re.sub(' +', ' ', processed)
                
        # Write processed text to new file
        output_filename = 'processed_' + filename
        output_path = output_dir / output_filename
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(processed)
            
        print(f"Successfully processed {filename}")
            
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")


#### 1.2 Iteration to process all files in the "unprocessedTexts" folder

In [9]:
# Process all files in the unprocessedTexts folder
input_dir = Path("unprocessedTexts")
for file in input_dir.iterdir():
    if file.is_file():
        process_text(file.name)

Successfully processed Anthem.txt
